# Clean and link filings



## 0. Imports and functions

In [22]:

import pandas as pd
import pickle
import re
import numpy as np
import plotnine
from plotnine import *


pd.set_option('display.float_format', lambda x: '%.3f' % x)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Load demographic data


In [23]:
df_crdc = pd.read_pickle("../../intermediate_objects/cleaned_df/dc_crdc_2013.pkl")
df_cc = pd.read_pickle("../../intermediate_objects/cleaned_df/dc_ccd_2013.pkl")

df_cc.head()
df_crdc.head()

df_crdc['ncesid_clean'] = df_crdc.NCESSCH.astype(str).str.replace("\\.", "", regex = True)

which_var,nces_name,which_year,frpl_eligible_rate,black_rate,white_rate,hispanic_rate
0,ACADEMY OF HOPE ADULT PCS,_2014-15,NaN,NaN,NaN,NaN
1,ACADEMY OF HOPE ADULT PCS,_2015-16,NaN,NaN,NaN,NaN
2,ACHIEVEMENT PREPARATORY PCS ELEMENTARY,_2013-14,0.987,1.000,0.000,0.000
3,ACHIEVEMENT PREPARATORY PCS ELEMENTARY,_2014-15,0.625,0.996,0.004,0.000
4,ACHIEVEMENT PREPARATORY PCS ELEMENTARY,_2015-16,0.538,0.978,0.000,0.022


,SCHOOL_NAME,STUDENTS_WITH_DISABILITIES_SERVED_UNDER_IDEA,STUDENTS_WITH_DISABILITIES_SERVED_UNDER_SECTION_504,total_students_iep_data,STUDENTS_WITH_DISABILITIES_SERVED_UNDER_SECTION_504_rate,STUDENTS_WITH_DISABILITIES_SERVED_UNDER_IDEA_rate,YEAR,NCESSCH,total_students_ressec_data,TOTAL_RESTRAINT_SECLUDE,TOTAL_RESTRAINT_SECLUDE_rate,merge_ieprestraints,total_students_disc_data,TOTAL_DISCIPLINE,TOTAL_DISCIPLINE_rate,merge_iepdisc
0,ACHIEVEMENT PREPARATORY ACADEMY PCS,NaN,7.000,355.000,0.020,NaN,2013.000,110007200405.000,382.000,0.000,0.000,both,382.000,205.000,0.537,both
1,ACHIEVEMENT PREPARATORY PCS-ELEMENTARY,NaN,2.000,222.000,0.009,NaN,2013.000,110007200473.000,233.000,0.000,0.000,both,233.000,20.000,0.086,both
2,AITON ES,26.000,4.000,249.000,0.016,0.104,2013.000,110003000122.000,247.000,0.000,0.000,both,247.000,57.000,0.231,both
3,AMIDON BOWEN ES,50.000,2.000,341.000,0.006,0.147,2013.000,110003000121.000,342.000,0.000,0.000,both,342.000,69.000,0.202,both
4,ANACOSTIA SHS,224.000,7.000,766.000,0.009,0.292,2013.000,110003000085.000,751.000,0.000,0.000,both,751.000,338.000,0.450,both


## 2. Load filings data

In [50]:
filings = pd.read_pickle("../../intermediate_objects/cleaned_df/dc_filings.pkl")
filings.type_closingorder.value_counts(normalize = True)

filings.

Order   0.635
HOD     0.365
Name: type_closingorder, dtype: float64

In [46]:
## first try merging between the nces school-level id and crdc

filings['ncesid_clean'] = np.where(filings.ncessch.notnull(),
                                   filings.ncessch.astype(str).str.replace("\\.", "", regex = True),
                                   np.nan)


## aggregate by non-missing ncesid
filings_agg = filings[filings.ncesid_clean.notnull()].groupby(['ncesid_clean']).agg({'case_no': 'nunique'}).reset_index()

## left join onto crdc
filings_agg_wcrdc = pd.merge(df_crdc,
                            filings_agg,
                            on = 'ncesid_clean',
                            how = 'left',
                            indicator = 'crdc_status')


filings_agg_complete = filings_agg_wcrdc[(filings_agg_wcrdc.STUDENTS_WITH_DISABILITIES_SERVED_UNDER_IDEA.notnull()) &
                                        (filings_agg_wcrdc.STUDENTS_WITH_DISABILITIES_SERVED_UNDER_IDEA_rate < 0.5)].copy()


filings_agg_complete['case_status'] = np.where(filings_agg_complete.crdc_status == "both",
                                              "Any filings",
                                              "No filings")

## write csv for R
filings_agg_complete.to_csv("../../intermediate_objects/cleaned_df/dc_filings_wcrdc.csv",
                           index = False)

In [ ]:
## where things left off:
## - fuzzy matching nces and ccd
## next steps:
## - load in matched data
## - get better ids
## - do left join on nces and hopefully more schools also have ccd data